In [6]:
import pandas as pd
import numpy as np

In [7]:
cidades = pd.read_excel('BRCidadesRegiao.xlsx')
cidades.replace([np.inf, -np.inf], np.nan, inplace=True)
cidades.dropna(inplace=True)
cidades_MG = cidades[cidades['estado'] == 'MG'].copy()

In [8]:
data = cidades_MG[
    [
        'regiao_imediata', 'cidade',
        'area_territorial', 'populacao_estimada', 'pib_per_capita',
        'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP'
    ]
]

In [9]:
variaveis = [
    'area_territorial', 'populacao_estimada', 'pib_per_capita',
    'saldo_receitas_despesas', 'IDHM_Educacao', 'GDP'
]

In [10]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [11]:
datav = data[variaveis].copy()
kmeans = KMeans(n_clusters=5, random_state=0).fit(datav)
datac = data[['regiao_imediata', 'cidade']].copy()
datac = datac.merge(datav, left_index=True, right_index=True)
datac['cluster'] = kmeans.labels_
datas = datac.sort_values(by='cluster')
datas

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,regiao_imediata,cidade,area_territorial,populacao_estimada,pib_per_capita,saldo_receitas_despesas,IDHM_Educacao,GDP,cluster
3260,Monte Carmelo,Abadia dos Dourados,881.064,7006,21953.45,3101.37187,0.563,180.09,0
3809,Alfenas,Paraguaçu,424.296,21605,22067.09,3267.12784,0.623,502116.72,0
3810,Pouso Alegre,Paraisópolis,331.238,21221,27566.08,4883.20852,0.657,416347.02,0
3811,AP de Sete Lagoas/MG,Paraopeba,625.623,24700,23143.51,8793.66825,0.625,523838.98,0
3812,Itabira,Passabém,94.183,1633,11386.19,1822.06887,0.533,18927.53,0
...,...,...,...,...,...,...,...,...,...
3778,AP de Belo Horizonte/MG,Nova Lima,429.313,96157,98855.84,77455.00068,0.704,7480877.71,4
3615,AP de Ipatinga/MG,Ipatinga,164.884,265409,36993.39,48731.38751,0.705,8482789.91,4
4045,AP de Sete Lagoas/MG,Sete Lagoas,536.928,241835,34977.38,52739.04098,0.705,7270911.68,4
3506,Divinópolis,Divinópolis,708.115,240408,25695.97,69636.66499,0.702,5635252.84,4


In [12]:
datac.pivot_table(index='regiao_imediata', columns='cluster',
                  values='area_territorial', aggfunc='count', fill_value=0)

cluster,0,1,2,3,4
regiao_imediata,,,,,
AP de Além Paraíba/MG - Sapucaia/RJ,3,0,0,0,0
AP de Araxá/MG,6,0,0,0,1
AP de Barbacena/MG,16,0,1,0,0
AP de Belo Horizonte/MG,40,0,10,2,1
AP de Brasília/DF,2,0,1,0,0
...,...,...,...,...,...
Uberaba,7,1,1,0,0
Uberlândia,12,0,1,0,0
Unaí,11,0,0,0,0


In [13]:
centroides = kmeans.cluster_centers_
centroides = pd.DataFrame(
    centroides,
    columns=variaveis
)
centroides = centroides.reset_index()
centroides = centroides.rename(columns={'index': 'cluster'})
centroides

,cluster,area_territorial,populacao_estimada,pib_per_capita,saldo_receitas_despesas,IDHM_Educacao,GDP
0,0,649.776914,13620.883184,17093.682863,4162.012816,0.547107,1.496252e+05
1,1,2979.853000,455188.500000,34210.695000,58077.339045,0.708000,1.399327e+07
2,2,1324.893333,91149.750000,33077.546250,26858.948908,0.659667,2.199073e+06
3,3,269.315000,556866.500000,49034.060000,246194.255485,0.692000,2.581592e+07
4,4,525.892000,160654.333333,66012.075556,69749.384429,0.697222,6.358926e+06


In [14]:
from pyod.models.knn import KNN

In [16]:
sc_x = StandardScaler()
datax = sc_x.fit_transform(datav)
clf = KNN().fit(datav)
outliers = clf.predict(datav)
dataod = data.copy()
dataod['outlier'] = outliers
dataod


,regiao_imediata,cidade,area_territorial,populacao_estimada,pib_per_capita,saldo_receitas_despesas,IDHM_Educacao,GDP,outlier
3260,Monte Carmelo,Abadia dos Dourados,881.064,7006,21953.45,3101.37187,0.563,180.09,0
3261,Divinópolis,Abaeté,1817.067,23250,18044.21,4446.10517,0.556,430235.36,0
3262,Manhuaçu,Abre Campo,470.551,13444,13453.99,2522.55424,0.525,187903.74,0
3263,AP de Ponte Nova/MG,Acaiaca,101.886,3994,9535.73,1627.27342,0.499,36424.03,0
3264,AP de Ipatinga/MG,Açucena,815.422,9368,9702.71,-35.68340,0.451,94656.19,0
...,...,...,...,...,...,...,...,...,...
4104,Guanhães,Virginópolis,439.878,10484,12967.44,2681.38291,0.580,136557.34,0
4105,Governador Valadares,Virgolândia,281.022,5340,8943.63,1975.47900,0.497,47823.28,0
4106,AP de Ubá/MG,Visconde do Rio Branco,243.351,42965,26962.77,817.95123,0.591,1052892.67,1
4107,AP de Além Paraíba/MG - Sapucaia/RJ,Volta Grande,205.552,5261,11865.03,2652.94812,0.531,79674.48,0


In [17]:
dataod = dataod[dataod['outlier'] == 1].copy()

In [19]:
dataod

,regiao_imediata,cidade,area_territorial,populacao_estimada,pib_per_capita,saldo_receitas_despesas,IDHM_Educacao,GDP,outlier
3275,Alfenas,Alfenas,850.446,80494,33422.98,24254.70415,0.712,2294655.05,1
3297,Uberlândia,Araguari,2729.508,117825,34840.91,32184.68529,0.716,3923828.15,1
3300,AP de Itumbiara/GO,Araporã,295.837,6931,175526.34,17330.82980,0.646,1081826.98,1
3303,AP de Araxá/MG,Araxá,1164.062,107337,49299.60,67160.00729,0.709,4761350.29,1
3320,AP de Barbacena/MG,Barbacena,759.186,138204,19631.86,23032.98697,0.687,2565190.32,1
...,...,...,...,...,...,...,...,...,...
4078,Uberaba,Uberaba,4523.957,337092,40066.32,109740.36611,0.705,13453594.31,1
4079,Uberlândia,Uberlândia,4115.206,699097,50548.78,133286.22264,0.716,32536.26,1
4089,Varginha,Varginha,395.396,136602,40506.11,74382.26005,0.707,4482211.77,1
4099,AP de Viçosa/MG,Viçosa,299.418,79388,19869.94,10317.63206,0.696,1509847.59,1


In [18]:
from sklearn.neighbors import NearestNeighbors

In [20]:
datao = dataod[variaveis].copy().reset_index()
datao = datao.drop(columns='index')
for cidade in dataod['cidade']:
    print(cidade)
    datax = dataod[dataod['cidade'] == cidade][variaveis]
    neighbors_alg = NearestNeighbors(n_neighbors=min(5, len(datao))).fit(datao)
    similar = neighbors_alg.kneighbors(datax, return_distance=False)[0]
    data_similar = dataod.iloc[similar][variaveis].reset_index()
    df_uf = dataod.iloc[similar]['cidade'].reset_index()
    df_uf = df_uf.merge(data_similar, on='index', how='left')
    df_uf = df_uf.drop(columns=['index'])
    print(df_uf)

Alfenas
           cidade  area_territorial  populacao_estimada  pib_per_capita   
0         Alfenas           850.446               80494        33422.98  \
1  João Monlevade            99.158               80416        32537.18   
2          Lavras           564.744              104783        24341.66   
3          Sabará           302.453              137125        18562.94   
4   Três Corações           828.038               80032        28056.22   

   saldo_receitas_despesas  IDHM_Educacao         GDP  
0              24254.70415          0.712  2294655.05  
1              19654.74800          0.708  2320529.70  
2              33218.79310          0.719  2269918.22  
3              44102.85518          0.670  2280778.58  
4              19416.18666          0.660  2214642.02  
Araguari
               cidade  area_territorial  populacao_estimada  pib_per_capita   
0            Araguari          2729.508              117825        34840.91  \
1      Patos de Minas          3190.45